In [ ]:
import os
import math

import matplotlib.pyplot as plt
import torchvision
import numpy as np
import torch
from HemaDataset import HemaDataset
from HemaModel import HemaModel
from DINOv2ForRadiology.dinov2.eval.segmentation.utils import UNetDecoder
from DINOv2ForRadiology.dinov2.data.transforms import make_segmentation_train_transforms, make_segmentation_eval_transforms

from monai.losses.dice import DiceLoss, DiceCELoss
from monai.metrics import DiceMetric

In [ ]:
def one_epoch(model, data_loader, loss_fn=None, optimizer=None, scheduler=None, eval_metric=None):
    
    is_training = model.training
    all_loss = []

    for image, mask in data_loader:

        image = image.cuda(non_blocking=True)
        mask = mask.cuda(non_blocking=True)

        output = model(image)

        if is_training:
            loss = loss_fn(output, mask)

            # compute the gradients
            optimizer.zero_grad()
            loss.backward()

            # step
            optimizer.step()
            scheduler.step()

            all_loss.append(loss.item())
        else:
            for image, mask in data_loader:
                image = image.cuda(non_blocking=True)
                mask = mask.cuda(non_blocking=True)

                output = model(image)

                eval_metric(y_pred=output, y=mask)
                
    if is_training:
        return ( sum(all_loss) / len(all_loss) ) # return average loss
    return eval_metric # else, return evaluation metric

def train(model, train_loader, val_loader, loss_fn, optimizer, scheduler=None, epochs=100):
    
    best_dice = 0
    for epoch in range(epochs):
        
        model.train()
        avrg_loss = one_epoch(model=model, data_loader=train_loader, loss_fn=loss_fn, optimizer=optimizer, scheduler=scheduler)    
        
        if epoch % 10 == 0:
        
            model.eval()
            print(f"epoch: {epoch}, loss {avrg_loss}, ", end="")
            eval_metric = DiceMetric(include_background=False, reduction="none") # bug with reduction "mean," will do it manually.

            eval_metric = one_epoch(model=model, data_loader=val_loader, eval_metric=eval_metric)

            dice = eval_metric.aggregate().mean(axis=1).mean(axis=0).item() # take average across classes first (channels), then across batch. 
            print(f"dice: {dice}") 

            if dice > best_dice:
                best_dice = dice
                best_model = model.state_dict()

    return best_model, best_dice

def tune_lr(lrs, model, train_loader, val_loader, loss_fn, optimizer, scheduler=None, epochs=100):
    results = {}
    for lr in lrs:
        _, dice = train(model=model, train_loader=train_loader, val_loader=val_loader, loss_fn=loss_fn, optimizer=optimizer,
                        scheduler=scheduler, epochs=epochs)
        results[lr] = dice
    return results

In [ ]:
image_size = 224
root = f"{os.getcwd() + os.sep}data{os.sep}Dataset011_Cell"

train_image_transform, train_target_transform  = make_segmentation_train_transforms(resize_size=image_size)
eval_image_transform, eval_target_transform  = make_segmentation_eval_transforms(resize_size=image_size)

train_dataset = HemaDataset(root=root, split="train", seg_entire_cell=True, image_transform=train_image_transform, target_transform=train_target_transform)
val_dataset = HemaDataset(root=root, split="val", seg_entire_cell=True, image_transform=eval_image_transform, target_transform=eval_target_transform)

In [ ]:
batch_size = 1
epochs = 100
epoch_length = math.ceil(len(train_dataset) / batch_size)
max_iter = epoch_length * epochs 

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

encoder = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').cuda()
decoder = UNetDecoder(in_channels=encoder.embed_dim, out_channels=3, image_size=224, resize_image=True).cuda()
model = HemaModel(encoder=encoder, decoder=decoder)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)    

optim_param_groups = [
    {"params": encoder.parameters(), "lr": 1e-5},
    {"params": decoder.parameters(), "lr": 1e-2}
]

loss_fn = DiceCELoss(sigmoid=True)
optimizer = torch.optim.SGD(optim_param_groups, momentum=0.9, weight_decay=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_iter, eta_min=0)

In [ ]:
lrs = [5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 1e-6, 5e-7, 1e-7]
results = {}

train(model=model, train_loader=train_loader, val_loader=val_loader,
        loss_fn=loss_fn, optimizer=optimizer, scheduler=scheduler, epochs=epochs)